### Import lib


In [1]:
import os, json
#from langchain.chains import AnalyzeDocumentChain
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_openai import ChatOpenAI

from utils import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY 
from llm_utils import load_pdf_documets, RecursiveCharacterTextSplitter
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

#openai.api_key = os.environ['OPENAI_API_KEY']

[nltk_data] Downloading package punkt_tab to /home/oleg/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
from langchain_openai import ChatOpenAI
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, top_p=0.2)  

/home/oleg/miniconda3/envs/llm/lib/python3.10/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


### Load documets

In [3]:
#loader_url =SeleniumURLLoader(["https://arxiv.org/pdf/2312.10997",
pdf_list = ["./data/data_rag/s41524-023-01062-z.pdf",
    "./data/data_rag/s41699-023-00369-1.pdf"
         #   "https://github.com/HSE-LAMBDA/ai4material_design/tree/main/docs/CONSTRUCTOR-MOCK.md"
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/CONSTRUCTOR.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/DATA.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/ENVIRONMENT.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-CONSTRUCTOR.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-MOCK.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/PILOT.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/SPARSE-PAPER.md"
          #  "https://www.nature.com/articles/s41377-024-01407-3",
          #  "https://www.nature.com/articles/s41565-023-01407-1",
          #  "https://www.nature.com/articles/s41699-023-00369-1",
           ]
                               
chunks_0, mat_documents = load_pdf_documets(pdf_list, tokens=2000)
print(len(chunks_0), len(mat_documents))

75 35


### Prompts

In [4]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_core.prompts import PromptTemplate

In [5]:
page_title = PromptTemplate(input_variables=['text'], template="""
            You are a distinguished expert in the field of scientific literature analysis. You have been provided with a page from a scientific article:
            ###
              {text}
            -------------
            Please identify the title of the article.
            The title is usually positioned prominently at the top of the page, often distinguished by a larger font size, bold typeface, or centered alignment. It is concise and encapsulates the main theme of the article.
            Typically, the authors' names appear directly below the title.\n
            
            The output format is a dictionary in valid JSON format.
            Where keys are 'yes': value - a Boolean value, True if a title is found, otherwise False,
               'title': value - the identified title of the article or an empty string if no title is found.
            
            Example: 
                'yes': true , 'title': 'Understanding the Mechanisms of Quantum Computing'
            
            Please ensure the output is precise and adheres to the specified format.
            """)

extract_author = PromptTemplate(input_variables=['text', 'title'], template="""
            You are a distinguished expert in the field of scientific literature analysis. You have been provided with a page from a scientific article:
            ###
                {text}
            ------------- \n
            And the tittle of the article: 
            ###
                {title}
            ------------- \n
            Your task is to analyze the given article and answer the question: 
            Who  are authors of the article and where was this study?
            Please note that usually, the authors are listed immediately after the title of the article 
            and positioned prominently at the top of the page. At the buttom of the page  there is a list of references not authors.
            And if names of the institutions where the author works are not listed next to the author, then it is listed in the reference number listed next to the author.
            The text of the article does not list the authors, but the works that the authors refer to in their article. \n
            
            The output format is a dictionary in valid JSON format.
            Where keys is an author names and values is a list of his institutions.
            
            Example: 
                Claudio Cazorla: ['Department of Physics, Universitat Politècnica de Catalunya, 08034 Barcelona, Spain'], 
                'Valeria Nicolosi': ['CRANN and AMBER, Dublin 2, Ireland', 'Trinity College Dublin, Dublin 2, Ireland'],
                ...  
            
            Please ensure the output is precise and adheres to the specified format.
            """
                )



extract_inst = PromptTemplate.from_template(
            """
            You are an expert in analyzing scientific papers.
            You have received the page of an article:
            ###
                {text}
            ------------- \n
            And the list of refereneces in the article as numbers: 
            ###
                {numbers}
            ------------- \n
            Your task is to analyze the given article and extact names of these  references. 
            Please note that usually, the references  are listed at the buttom of the page. 
            The text of the article does not list the authors, but the works that the authors refer to in their article. \n
            
            The output format is a dictionary in valid JSON format.
            Where key is a number of reference and value is a name of an institution.
            Example: 
                 '1': 'Institute for Functional Intelligent Materials, National University of Singapore',
                 '2': 'Constructor University Bremen gGmbH',
                 '3': 'HSE University',
                ...
                
            """
                )


In [6]:
import pathlib, pymupdf4llm, json

def print_dict(d, indent=0):
    """
    Recursively prints a dictionary, including nested dictionaries, 
    with indentation to represent structure.
    """
    for key, value in d.items():
        print('  ' * indent + str(key) + ": ", end="")
        if isinstance(value, dict):
            print()  # Print a newline for nested dictionaries
            print_dict(value, indent + 1)
        else:
            print(str(value))
            
def pdf2md(pdf_file):
    """
    Convert a PDF file to Markdown
    """
    
    md_file = pdf_file.replace('.pdf', '.md')
    if not pathlib.Path(md_file).exists():
        print("Convert PDF to MD")
        md_text = pymupdf4llm.to_markdown(pdf_file)
        # now work with the markdown text, e.g. store as a UTF8-encoded file
        pathlib.Path(md_file).write_bytes(md_text.encode())
    return md_file




def check_inst(answer):
    
    for author, institutions in answer.items():
        # Check if any institution is a string that is not a number
        has_institution_name = any(isinstance(inst, str) and not inst.isdigit() for inst in institutions)
        if not has_institution_name:
            return True
    return False
    

def get_inst(text, answer, 
             llm=ChatOpenAI(model="gpt-4o-mini",  temperature=0.0)):
    """
    Get the institution name from the text using reference numbers.
    """
    def str2int(s):
        try:
            return int(s)
        except ValueError:
            return s
    
    print("Get_Inst") 
    if not len(answer):
        return {}
    
    references = {}
    _rl = []
    for a,v in answer.items():
            #v = v.split(',')
        references[a] = [str2int(x) for x in v] 
        _rl += [x for x in v  if isinstance(x, int)]
    _rl = list(set(_rl))
    # Extract  the institution name from the text by reference numbers.
    inst = extract_inst | llm | JsonOutputParser()
    inst_dict = inst.invoke({'text': text, 'numbers':_rl})
    for a,v in references.items():
        references[a] = [inst_dict.get(str(x), x) for x in v ]
        
    return references

In [7]:
def get_author(doc,
               use_md = True,  # Use Markdown or not to  extract the title
               max_N_title = 3,  # Maximum number of attempt to find the title
               max_N_authors = 3, # Maximum number of attempt to find the authors
               check = True,   # Check if the institution name is in the dictionary or it has only number of reference or an institution.
    ):
    """
    Extracts the author from the text
    """
    author = False
    
    ollm = ChatOpenAI(model="gpt-4o",  temperature=0.0)
    llm = ChatOpenAI(model="gpt-4o-mini",  temperature=0.0)
    
    # What is  the title of the article on the page? Use LLM.
    title = page_title | ollm | JsonOutputParser()
    # Extract authors of the text by LLM.
    authors = extract_author | llm | JsonOutputParser()
    
    if use_md: #  Use Markdown to  extract the title
        pdf_file = doc.metadata['source'] 
        md_file = pdf2md(pdf_file)#  Convert the PDF file to Markdown
        md_doc = TextLoader(md_file).load()[0] #  Load the Markdown file
    else:
        md_doc = doc.copy()

    _f = True
    while _f: # First extract a title of the article. If not found, try again
        try:
            _title = title.invoke({'text':md_doc.page_content})
            _f = not _title['yes']
        except Exception as e:
            print(e)
        
        max_N_title -=1
        _f = False if  max_N_title < 0 else True
    
    if _title['yes']: 
        # Add metadata to a document.
        doc.metadata['title'] = _title['title']
        
        _f = True
        while _f: # Second  extract the authors if thre is a title.  If not found, try again
            try:
                answer = authors.invoke({'text':doc.page_content, 'title':_title['title']})
                _f = False if len(answer) > 0 else True
                author = not _f 
                
                #  Check or Not the institution name in the dictionary or it has only number of reference.
                check = check_inst(answer) if check else False 
                # Add metadata to a document.
                if check:
                    print(check, "check")
                    doc.metadata['authors'] = get_inst(doc.page_content, answer, llm)
                else:
                    doc.metadata['authors'] = answer if author  else {}     
            except Exception as e:
                print(e)
                
            max_N_authors -=1
            _f = False if  max_N_authors < 0 else True
    
    return author

In [8]:
ollm = ChatOpenAI(model="gpt-4o",  temperature=0.0)

# Is  the title of the article on the page?
title = page_title | ollm | JsonOutputParser()
# Extract authors of the text
authors = extract_author | llm | JsonOutputParser()

for doc in mat_documents:
    _title = title.invoke({'text':doc.page_content})
    if _title['yes']:
        answer = authors.invoke({'text':doc.page_content, 'title':_title['title']})
        print_dict(_title)
        print_dict(answer)
        print()

yes: True
title: Sparse representation for machine learning the properties of defects in 2D materials
Nikita Kazeev: ['Department of Physics, National University of Singapore, Singapore', 'Institute of Physics, Kazan Federal University, Kazan, Russia']
Abdalaziz Rashid Al-Maeeni: ['Department of Physics, Khalifa University, Abu Dhabi, UAE']
Ignat Romanov: ['Department of Physics, Khalifa University, Abu Dhabi, UAE']
Maxim Faleev: ['Department of Physics, National University of Singapore, Singapore']
Ruslan Lukin: ['Department of Physics, National University of Singapore, Singapore']
Alexander Tormasov: ['Department of Physics, National University of Singapore, Singapore']
A. H. Castro Neto: ['Department of Physics, National University of Singapore, Singapore', 'Graphene Research Centre, National University of Singapore, Singapore']
Kostya S. Novoselov: ['Department of Physics, National University of Singapore, Singapore', 'National Graphene Institute, University of Manchester, Manchest

In [9]:
set_metadata = set()

for doc in mat_documents:
    if  doc.metadata['source'] not in  set_metadata:
        if get_author(doc):
            set_metadata.add(doc.metadata['source'])
            print_dict(doc.metadata)
            print()

source: ./data/data_rag/s41524-023-01062-z.pdf
page: 0
title: Sparse representation for machine learning the properties of defects in 2D materials
authors: 
  Nikita Kazeev: ['Department of Physics, National University of Singapore, Singapore', 'Institute of Physics, Kazan Federal University, Kazan, Russia']
  Abdalaziz Rashid Al-Maeeni: ['Department of Physics, University of Sharjah, Sharjah, UAE']
  Ignat Romanov: ['Department of Physics, University of Sharjah, Sharjah, UAE']
  Maxim Faleev: ['Institute of Physics, Kazan Federal University, Kazan, Russia']
  Ruslan Lukin: ['Institute of Physics, Kazan Federal University, Kazan, Russia']
  Alexander Tormasov: ['Institute of Physics, Kazan Federal University, Kazan, Russia']
  A. H. Castro Neto: ['Department of Physics, National University of Singapore, Singapore']
  Kostya S. Novoselov: ['Department of Physics, National University of Singapore, Singapore']
  Pengru Huang: ['Department of Physics, National University of Singapore, Sing

In [10]:
set_metadata = set()

for doc in mat_documents:
    if  doc.metadata['source'] not in  set_metadata:
        if get_author(doc,  use_md = False):
            set_metadata.add(doc.metadata['source'])
            print_dict(doc.metadata)
            print()

source: ./data/data_rag/s41524-023-01062-z.pdf
page: 0
title: Sparse representation for machine learning the properties of defects in 2D materials
authors: 
  Nikita Kazeev: ['Department of Physics, National University of Singapore, Singapore', 'Institute of Physics, Kazan Federal University, Kazan, Russia']
  Abdalaziz Rashid Al-Maeeni: ['Department of Physics, University of Sharjah, Sharjah, UAE']
  Ignat Romanov: ['Department of Physics, National University of Singapore, Singapore']
  Maxim Faleev: ['Department of Physics, National University of Singapore, Singapore']
  Ruslan Lukin: ['Department of Physics, National University of Singapore, Singapore']
  Alexander Tormasov: ['Department of Physics, National University of Singapore, Singapore']
  A. H. Castro Neto: ['Department of Physics, National University of Singapore, Singapore', 'Graphene Research Centre, National University of Singapore, Singapore']
  Kostya S. Novoselov: ['Department of Physics, National University of Singap

#### Load documents of conference

In [58]:

# Specify the directory
directory = './data/data_rag/conference/'

# List all files in the directory
pdf_files = [f for f in os.listdir(directory) if f.endswith('.pdf')]

# Print PDF file names
#print(pdf_list)
pdf_files = [os.path.join(directory, f) for f in pdf_files]


chunks, documents = load_pdf_documets(pdf_files, tokens=5000)
print(len(chunks), len(documents))

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong poi

163 163


### Extract authors of the text

In [59]:
N = 2
set_metadata = set()  # Set to keep track of unique source of  metadata

while N > 0 and (len(set_metadata) < len(pdf_files)):
    for i, doc in enumerate(documents):
        if  doc.metadata['source'] not in  set_metadata:
            try: 
                #f = get_author(doc)
                if get_author(doc):
                    print(i)
                    print_dict(doc.metadata)
                    set_metadata.add(doc.metadata['source'])
                    print()
            except Exception as e:
                print(e)
                #print(10*'-')
            #print(doc.page_content)
            print(20*'-')
    N -= 1
    print(f"Loop: {N}, with authors: {len(set_metadata)}, totall: {len(pdf_files)}")
    print()

0
source: ./data/data_rag/conference/45_AI4AM2024_Hakim_AMARA_8.pdf
page: 0
title: Unlocking 3D Nanoparticle Shapes from 2D HRTEM images: A Deep Learning Breakthrough
authors: 
  R. Moreau: ['LEM (ONERA -CNRS), Châtillon, France']
  H. Amara: ['LEM (ONERA -CNRS), Châtillon, France', 'MPQ, Paris, France']
  M. Moreaud: ['IFP Energies Nouvelles, Solaize, France']
  J. Nelayah: ['MPQ, Paris, France']
  A. Moncomble: ['MPQ, Paris, France']
  D. Alloyeau: ['MPQ, Paris, France']
  C. Ricolleau: ['MPQ, Paris, France']
  R. Gatti: ['LEM (ONERA -CNRS), Châtillon, France']

--------------------
1
source: ./data/data_rag/conference/35_AI4AM2024_Cheng.pdf
page: 0
title: Predicting material properties with the help of machine learning
authors: 
  Bingqing Cheng: ['UC Berkeley, USA']

--------------------
--------------------
3
source: ./data/data_rag/conference/4_AI4AM2024_Magdau_Ioan_Bogdan_39.pdf
page: 0
title: Foundational MLIP: the Li-ion Battery
authors: 
  Ioan-Bogdan Magdău: ['Newcastle Univ

24
source: ./data/data_rag/conference/63_AI4AM2024_Seoane_Juan_Jose__83.pdf
page: 0
title: Exploring Neural Networks for Predicting Bohmian Trajectories in Many-Body Scenarios
authors: 
  Juan Jose Seoane: ['Universidad Autonoma de Barcelona, 08193 Bellaterra, SPAIN']
  Xavier Cartoixà: ['Universidad Autonoma de Barcelona, 08193 Bellaterra, SPAIN']
  Xavier Oriols: ['Universidad Autonoma de Barcelona, 08193 Bellaterra, SPAIN']

--------------------
25
source: ./data/data_rag/conference/16_AI4AM2024_Tang_Jialiang_45.pdf
page: 0
title: Exploring Ground States of Fermi Hubbard Model on Honeycomb Lattices with Counterdiabaticity
authors: 
  Jialiang Tang: ['Department of Physical Chemistry, University of the Basque Country UPV/EHU, Apartado 644, 48080 Bilbao', 'EHU Quantum Center, University of the Basque Country UPV/EHU, Barrio Sarriena, s/n, 48940 Leioa, Biscay, Spain']
  Ruoqian Xu: ['Department of Physical Chemistry, University of the Basque Country UPV/EHU, Apartado 644, 48080 Bilbao'

46
source: ./data/data_rag/conference/86_AI4AM2024_Garrido_Aldea_Jaime_85.pdf
page: 0
title: Development of an automated workflow for well converged DFT calculations for using SIESTA and the AIIDA infrastructure satisfying FAIR data principles
authors: 
  Jaime Garrido Aldea: ['ICN2, Instituto Catalan de Nanociencia y Nanotecnología UAB Campus, Bellaterra, 08193, Spain']
  Jose Hugo García Aguilar: ['ICN2, Instituto Catalan de Nanociencia y Nanotecnología UAB Campus, Bellaterra, 08193, Spain']
  Stephan Roche: ['ICREA — Institució Catalana de Recerca i Estudis Avançats, 08010 Barcelona, Spain']

--------------------
48
source: ./data/data_rag/conference/42_AI4AM2024_Grzelczak_Marek_34.pdf
page: 0
title: Machine Learning for Nanoparticle Synthesis
authors: 
  Marek Grzelczak: ['Centro de Física de Materiales (CSIC -UPV/EHU), Manuel Lardizabal 5, 20018 Donostia -San Sebastian', 'Donostia International Physics Center (DIPC), Manuel Lardizabal 4, 20018 Donostia -San Sebastian']
  Anish Rao

70
source: ./data/data_rag/conference/56_AI4AM2024_Toher_Cormac_17.pdf
page: 0
title: Predicting the synthesizability and properties of disordered materials by combining first-principles calculations with machine-learning
authors: 
  Cormac Toher: ['Department of Materials Science and Engineering, The University of Texas at Dallas, Richardson, TX 75080, USA']

--------------------
72
source: ./data/data_rag/conference/49_AI4AM2024_Cao_Junhao_33.pdf
page: 0
title: Denoising of 4D-STEM Dataset using Pix2Pix GAN and Artifact Reduction
authors: 
  Junhao Cao: ['Laboratoire de Réactivité et Chimie des Solides (LRCS), CNRS UMR 7314, Université de Picardie Jules Verne, Amiens, France', 'Réseau sur le Stockage Electrochimique de l’Energie (RS2E), CNRS FR 3459, Amiens, France']
  Nicolas Folastre: ['Laboratoire de Réactivité et Chimie des Solides (LRCS), CNRS UMR 7314, Université de Picardie Jules Verne, Amiens, France', 'Réseau sur le Stockage Electrochimique de l’Energie (RS2E), CNRS FR 3459,

83
source: ./data/data_rag/conference/30_AI4AM2024_Barnard_Amanda_49.pdf
page: 0
title: Higher-Order Pattern Recognition for Materials Informatics using Explainable Artificial Intelligence
authors: 
  Amanda S Barnard: ['School of Computing, Australian National University, 145 Science Road, Acton, Australia']
  Tommy Liu: ['School of Computing, Australian National University, 145 Science Road, Acton, Australia']

--------------------
84
source: ./data/data_rag/conference/14_AI4AM2024_Tomut_Andrei_41.pdf
page: 0
title: LatMatcher - AI-Powered Tool for 2D Material Stacking and Property prediction.
authors: 
  Andrei Voicu Tomut: ['Catalan Institute of Nanoscience and Nanotechnology (ICN2), CSIC and The Barcelona Institute of Science and Technology, Campus UAB, 08193 Bellaterra, Catalonia, Spain']
  Stephan Roche: ['ICREA — Institució Catalana de Recerca i Estudis Avançats, 08010 Barcelona, Spain']
  Jose Hugo Garcia Aguilar: ['Catalan Institute of Nanoscience and Nanotechnology (ICN2), C

104
source: ./data/data_rag/conference/52_AI4AM2024_Hernandez_Bertran_Michael_Alejandro_32.pdf
page: 0
title: An X-ray spectra simulations workflow based on machine learning: applications to Li-ion battery materials
authors: 
  Michael A. Hernandez Bertran: ['Istituto Nanoscienze, Consiglio Nazionale delle Ricerche CNR, via Campi 213/A 41125, Modena, Italy', 'FIM, University of Modena and Reggio Emilia, via Campi 213/A 41125, Modena, Italy']
  Davide Tisi: ['Institut des Sciences et Génie Matériaux, Ecole Polytechnique Fédérale de Lausanne (EPFL), 1015 Lausanne, Switzerland']
  Yusuf Shaidu: ['Department of Physics, University of California Berkeley, Berkeley, CA 94720, USA', 'Materials Sciences Division, Lawrence Berkeley National Laboratory, Berkeley, CA 94720, USA']
  Franco Pellegrini: ['Scuola Internazionale Superiore di Studi Avanzati, Trieste, Italy']
  Stefano de Gironcoli: ['Scuola Internazionale Superiore di Studi Avanzati, Trieste, Italy', 'CNR-IOM DEMOCRITOS, Istituto Offic

128
source: ./data/data_rag/conference/22_AI4AM2024_Kazeev_Nikita_78.pdf
page: 0
title: WyckoffTransformer: Autoregressive Generation of Crystals
authors: 
  Nikita Kazeev: ['Institute for Functional Intelligent Materials, National University of Singapore, Singapore']
  Andrey Ustyuzhanin: ['Constructor University Bremen gGmbH, Bremen, Germany']
  Ignat Romanov: ['HSE University, Moscow, Russia']
  Ruiming Zhu: ['School of Materials Science and Engineering, Nanyang Technological University, Singapore', 'Institute of Materials Research and Engineering, Agency for Science, Technology and Research (A*STAR), Singapore']
  Wei Nong: ['School of Materials Science and Engineering, Nanyang Technological University, Singapore', 'Institute of Materials Research and Engineering, Agency for Science, Technology and Research (A*STAR), Singapore']
  Shuya Yamazaki: ['School of Materials Science and Engineering, Nanyang Technological University, Singapore', 'Institute of Materials Research and Enginee

151
source: ./data/data_rag/conference/80_AI4AM2024_Csanyi.pdf
page: 0
title: A foundational atomistic model for materials
authors: 
  Gábor Csányi: ['Engineering Laboratory, University of Cambridge, Trumpington St, Cambridge, United Kingdom']

--------------------
152
source: ./data/data_rag/conference/90_AI4AM2024_Schleder_Gabriel_42.pdf
page: 0
title: Performance Assessment of Universal Machine Learning Interatomic Potentials: Challenges and Directions for Materials' Surfaces
authors: 
  Bruno Focassio: ['Brazilian Nanotechnology National Laboratory (LNNano/CNPEM), 13083-100, Campinas, SP, Brazil']
  Luis Paulo M Freitas: ['Brazilian Nanotechnology National Laboratory (LNNano/CNPEM), 13083-100, Campinas, SP, Brazil']
  Gabriel R. Schleder: ['Brazilian Nanotechnology National Laboratory (LNNano/CNPEM), 13083-100, Campinas, SP, Brazil', 'John A. Paulson School of Engineering and Applied Sciences, Harvard University, Cambridge, Massachusetts 02138, USA']

--------------------
154
sourc

In [60]:
print_dict(doc.metadata)

source: ./data/data_rag/conference/89_AI4AM2024_Botti.pdf
page: 0
title: The future is here: accelerating computational materials science with machine learning
authors: 
  Silvana Botti: ['Research Center Future Energy Materials and Systems, Faculty of Physics and Astronomy and ICAMS, Ruhr University Bochum, Germany']


In [12]:
len(set_metadata), len(pdf_files)

(91, 92)

In [17]:
doc.copy()

Document(page_content='A I 4 AM 2 02 4  J u l y  0 2 -0 4 ,  2 0 2 4  - B a r c e l o n a  ( S p a i n )  \n The future is here: accelerating \ncomputational materials science with \nmachine learning  \n \nSilvana Botti  \nResearch Center Future Energy Materials and Systems  \nFaculty of Physics and Astronomy and ICAMS  \nRuhr University Bochum, Germany  \n \nsilvana.botti@rub.de  \n \n \n \nIn this presentation we explore the use of materials \ndatabases [1] to design new functional materials \nwith a focus on energy applications. The \ncombination of high -throughput techniques and \nrapidly advancing supercomputers enables the \nautomated screening of large numbers of \nhypothetical materials, providing solutions to current \ntechnological challenges. In addition, the integration \nof machine learning methods with density functional \ntheory offers a powerful approach to accelerate \nmaterials discovery. We summarise our recent \nefforts in the discovery, characterisation and \nunde

#### Where there are not exctracted authors

In [62]:
not_found = []

for doc in documents:
    if  doc.metadata['source'] not in  set_metadata:
        try: 
            
            print("Authors :",doc.metadata['authors'])
            print(10*'-')
            print(doc.page_content)
            print()
        except Exception as e:
            print(e)
            print(10*'-')
            print(doc.metadata['source'])
            not_found.append(doc.metadata['source'])
            print()
            print(doc.page_content)
            print()
            #_title = title.invoke({'text':doc.page_content})
            #print(_title)
            print(10*'-')

'authors'
----------
./data/data_rag/conference/78_AI4AM2024_Pinto_Ivan_57_2.pdf

A I 4 AM 2 02 4  J u l y  02-04,  2 0 24 - B a r c e l o n a  ( S p a i n )  
 To be defined  
 
Ivan Pinto  
1ICN2, Spain  
 
ivan.pinto@icn2.cat

----------


In [39]:
for doc in documents:
    if  doc.metadata['source'] in  set_metadata:
        try: 
            
            print("Authors :",doc.metadata['authors'])
            print(10*'-')
            #print(doc.page_content[:1000])
            print()
        except Exception as e:
            print(e)
            print(10*'*** ')
            print(doc.metadata['source'])
            print()
            print(doc.page_content[:200])
            print()
            #_title = title.invoke({'text':doc.page_content})
            #print(_title)
            print(10*'-')

Authors : {'R. Moreau': ['LEM (ONERA -CNRS), Châtillon, France'], 'H. Amara': ['LEM (ONERA -CNRS), Châtillon, France', 'MPQ, Paris, France'], 'M. Moreaud': ['IFP Energies Nouvelles, Solaize, France'], 'J. Nelayah': ['MPQ, Paris, France'], 'A. Moncomble': ['MPQ, Paris, France'], 'D. Alloyeau': ['MPQ, Paris, France'], 'C. Ricolleau': ['MPQ, Paris, France'], 'R. Gatti': ['LEM (ONERA -CNRS), Châtillon, France']}
----------

Authors : {'Bingqing Cheng': ['UC Berkeley, USA']}
----------

'authors'
*** *** *** *** *** *** *** *** *** *** 
./data/data_rag/conference/4_AI4AM2024_Magdau_Ioan_Bogdan_39.pdf

AI4AM2024 July 02-04, 2024 - Barcelona (Spain)

----------
Authors : {'Ioan-Bogdan Magdău': ['Newcastle University, NE1 7RU, UK'], 'I Batatia': ['University of Cambridge, CB2 1TN, UK'], 'P Benner': ['BAM, D-12205, Germany'], 'Y Chiang': ['University of California Berkeley, CA 94720, USA', 'LBNL, CA 94720, USA'], 'AM Elena': ['STFC, WA4 4AD, UK'], 'DP Kovács': ['University of Cambridge, CB2 1TN

In [69]:
def get_document(path):
    
    for d in documents:
        if d.metadata['source'] == path:
            return d

for d in not_found:
    print(d)
    print()
    d = get_document(d)
    print(d.page_content)
    print(10*'-')

./data/data_rag/conference/78_AI4AM2024_Pinto_Ivan_57_2.pdf

A I 4 AM 2 02 4  J u l y  02-04,  2 0 24 - B a r c e l o n a  ( S p a i n )  
 To be defined  
 
Ivan Pinto  
1ICN2, Spain  
 
ivan.pinto@icn2.cat
----------


#### Save Document's metadata  to a JSON file

In [15]:
meta_data = []  # List to keep track of metadata
for d in documents:
    meta_data.append(d.metadata)

with open ('./data/data_rag/conference/conference.json', 'w') as f: 
    json.dump(meta_data, f, indent=4) 

